In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
#Log in to Huggingface
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)nstruct-sharded/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)ded/resolve/main/configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


(…)-sharded/resolve/main/modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)rded/resolve/main/generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)arded/resolve/main/tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)ruct-sharded/resolve/main/tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

(…)ded/resolve/main/special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Prep dataset

In [ ]:
import pandas as pd
import gdown
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint

from gensim.models.coherencemodel import CoherenceModel


file_id = "1u7rvNf84a__E3HaPfhcC83MLENOieBZe"
url = f'https://drive.google.com/uc?id={file_id}'
output = 'data.csv'
gdown.download(url, output, quiet=False)
df = pd.read_csv('data.csv')
filtered_df = df[df['DocID'].str.startswith('S')]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Downloading...
From: https://drive.google.com/uc?id=1u7rvNf84a__E3HaPfhcC83MLENOieBZe
To: /content/data.csv
100%|██████████| 1.31G/1.31G [00:07<00:00, 175MB/s]


In [ ]:
pharm_df = pd.read_csv('pharm_data.csv')
pharm_df

,Abstract,BodyText,DocID,Title,novel_concepts
0,The role of early coronary revascularization i...,The present study differs from these previous ...,S0002914916310487,Impact of Early Coronary Revascularization on ...,The novel concepts presented in this work incl...
1,Recovery in heart rate (HR) after exercise is ...,Peak HR was higher in controls compared with t...,S0002914913024648,Heart rate recovery in patients with hypertrop...,The novel concepts presented in this work at t...
2,Polygenic scores are a popular tool for predic...,"However, only for the educational attainment G...",S0002929719302319,Comparing Within- and Between-Family Polygenic...,The novel concepts presented in this work incl...
3,The exosome is a conserved multi-protein compl...,"However, in fibroblasts from individuals with ...",S0002929718301009,Variants in EXOSC9 Disrupt the RNA Exosome and...,The novel concepts presented in this work incl...
4,Sequencing the genomes of extinct hominids has...,We compare it to the human and chimpanzee refe...,S0002929716300337,The Divergence of Neandertal and Modern Human ...,The novel concepts presented in this work incl...
...,...,...,...,...,...
86,To identify further Mendelian causes of intell...,We compared this number to the observed number...,S0002929714001086,"De novo loss-of-function mutations in SETD5, e...",The novel concepts presented in this work incl...
87,Zoroastrianism is one of the oldest extant rel...,These values were then fixed before running CH...,S0002929717302914,The Genetic Legacy of Zoroastrianism in Iran a...,The novel concepts presented in this work incl...
88,UBE2L3 is associated with increased susceptibi...,HOIL-1-deficient mice have defective NF-κB res...,S0002929715000038,UBE2L3 polymorphism amplifies NF-κB activation...,The novel concepts presented in this work invo...
89,Bacteremia (bacterial bloodstream infection) i...,Total RNA was extracted with the RNeasy mini k...,S000292971630060X,Polymorphism in a lincRNA Associates with a Do...,The novel concepts presented in this work incl...


### Create a training data set that finds initial topics via ChatGPT

### Prep data for model


In [ ]:
prepend_text = "Please describe the novel concepts presented in this work at the collegiate level: "
pharm_df['Prompt'] = pharm_df['BodyText'].apply(lambda x: prepend_text + x)

In [ ]:
from datasets import Dataset

data = Dataset.from_pandas(pharm_df)

In [ ]:
def generate_prompt(data):
  return f"""
<human>: {data["Prompt"]}
<assistant>: {data["novel_concepts"]}
""".strip()

def generate_and_tokenize_prompt(data):
  full_prompt = generate_prompt(data)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

# Test on original model

In [ ]:
prompt = """
<human>: "You will help me perform topic modeling on a scientific article. Without context, please answer: what is the most relevant scientific field discussed in the text (if ambiguous, give the most likely answer)"
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 5
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
#%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
#predict

def predict(df):
  device = "cuda:0"

  for index, row in df.iterrows():
    print(f'processing row: {index+1}')
    prompt = row['Prompt']
    prompt = prompt[:2048]
    #print(prompt)
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids = encoding.input_ids,
          attention_mask = encoding.attention_mask,
          generation_config = generation_config
      )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
      response = response.split(prompt)[1]
      #topic = response.split("The most relevant scientific field discussed in this text is ")[1]
    except:
      pass
    topic = response
    print(f'topic: {topic}')
    df.loc[index, 'Topic'] = topic
    response = '' #clear after each loop in case of errors
  return df

In [ ]:
untrained_data = topic_training_df.copy()
untrained_data.loc[:, 'Topic'] = ''
untrained_predictions = predict(untrained_data)

In [ ]:
untrained_data.to_csv('untrained_data.csv', index=False)

In [ ]:


device = "cuda:0"

for index, row in untrained_data.iterrows():
  print(f'processing row: {index+1}')
  prompt = row['Prompt']
  prompt = prompt[:2048]
  print(prompt)
  encoding = tokenizer(prompt, return_tensors="pt").to(device)
  with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  response = response.split(prompt)[1]
  try:
    topic = response.split("The most relevant scientific field discussed in this text is ")[1]
  except:
    topic = response
  print(topic)
  df.loc[index, 'Topic'] = topic
  response = '' #clear after each loop in case of errors

In [ ]:
untrained_data

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Step,Training Loss
1,2.739200
2,2.664800
3,2.676400
4,2.301600
5,2.443300
6,2.650300
7,2.546500
8,2.500800
9,2.362000
10,2.260100


TrainOutput(global_step=22, training_loss=2.4830016547983345, metrics={'train_runtime': 735.8908, 'train_samples_per_second': 0.124, 'train_steps_per_second': 0.03, 'total_flos': 3587893301870592.0, 'train_loss': 2.4830016547983345, 'epoch': 0.97})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "CodeChemist/Scientific_Topic_Modeling"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CodeChemist/Scientific_Topic_Modeling/commit/c3348dc7e21dfa4af5febdbc6d46abfb0419a81c', commit_message='Upload model', commit_description='', oid='c3348dc7e21dfa4af5febdbc6d46abfb0419a81c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
'''
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "vilsonrodrigues/falcon-7b-instruct-sharded" #path/to/your/model/or/name/on/hub"
adapter_model_name = "CodeChemist/Scientific_Topic_Modeling"

model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(model, adapter_model_name)
'''

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:7                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:461 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   458 │   │   │   if kwargs.get("torch_dtype", None) == "auto":                                  │
│   459 │   │   │   │   _ = kwargs.pop("torch_dtype")                                              │
│   460 │   │   │                                                                                  │
│ ❱ 461 │   │   │   config, kwargs = AutoConfig.from_pretrained(                                   │
│   462 │   │   │   │   pretrained_model_name_or_path,                                             │
│   463 │   │   │   │   return_unused_kwargs=True,                                                 │
│   464 │   │   │   │   trust_remote_code=trust_remote_code,                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:937 in    │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   934 │   │   config_dict, unused_kwargs = PretrainedConfig.get_config_dict(pretrained_model_n   │
│   935 │   │   if "auto_map" in config_dict and "AutoConfig" in config_dict["auto_map"]:          │
│   936 │   │   │   if not trust_remote_code:                                                      │
│ ❱ 937 │   │   │   │   raise ValueError(                                                          │
│   938 │   │   │   │   │   f"Loading {pretrained_model_name_or_path} requires you to execute th   │
│   939 │   │   │   │   │   " repo on your local machine. Make sure you have read the code there   │
│   940 │   │   │   │   │   " set the option `trust_remote_code=True` to remove this error."       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Loading vilsonrodrigues/falcon-7b-instruct-sharded requires you to execute the configuration file in 
that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option
`trust_remote_code=True` to remove this error.

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:270 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   267 │   </Tip>                                                                                 │
│   268 │   """                                                                                    │
│   269 │   try:                                                                                   │
│ ❱ 270 │   │   response.raise_for_status()                                                        │
│   271 │   except HTTPError as e:                                                                 │
│   272 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   273                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:1021 in raise_for_status              │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/None/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:417 in cached_file             │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                              

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: midjourney prompt for a boy running in the snow
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))